In [4]:
# !pip install tensorflow==1.15.0

# %tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.0


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np


In [52]:
# Caricamento dei dati
X = np.load('./dataset/states.npy')
y = np.load('./dataset/evals.npy')


# Creazione del modello
model = models.Sequential()

# Aggiunta di un layer di input (vettore di lunghezza 81)
model.add(layers.InputLayer(input_shape=(81,)))

# Layer hidden: possiamo utilizzare 2 o più layer densi
model.add(layers.Dense(128, activation='relu'))  # Primo layer denso con 128 neuroni
model.add(layers.Dense(64, activation='relu'))   # Secondo layer denso con 64 neuroni

# Layer di output: un singolo neurone con attivazione sigmoide per avere un output tra 0 e 1
model.add(layers.Dense(1, activation='sigmoid'))

# Compilazione del modello
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Riassunto del modello
model.summary()

# Addestramento del modello
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               10496     
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 65        
Total params: 18,817
Trainable params: 18,817
Non-trainable params: 0
_________________________________________________________________
Train on 17784 samples, validate on 4446 samples
Epoch 1/20
17784/17784 [==============================] - 1s 44us/sample - loss: 0.0039 - mean_absolute_error: 0.0480 - val_loss: 0.0036 - val_mean_absolute_error: 0.0479
Epoch 2/20
17784/17784 [==============================] - 1s 39us/sample - loss: 0.0026 - mean_absolute_error: 0.0397 - val_loss: 0.0041 - val_mean_absolute_err

In [7]:

# Previsione su nuovi dati
test_input = np.random.choice([0, 1, -1], size=(1, 81))  # Un esempio casuale di input
test_input[0, 14] = 2
prediction = model.predict(test_input)

print(f'test_input: {test_input}')
print(f'Predizione: {prediction[0][0]}')

test_input: [[ 1 -1  0 -1  0  1  0  0  1  1 -1  0 -1  1  2  1  0  1 -1  0  1 -1  0  0
   1  1  1 -1  1  0  1  1  0  1  0 -1 -1 -1  0  0  0  1  1  0  0  1  1 -1
   1  0  0  0  0  0  0 -1 -1  0  0  0 -1  0  1 -1  0 -1 -1  0  0 -1  1  1
   0  1 -1  1  1 -1  0 -1  1]]
Predizione: 0.8337529897689819


In [20]:
tf.compat.v1.disable_eager_execution()

# Carica il modello SavedModel
with tf.compat.v1.Session() as sess:
    tf.saved_model.loader.load(sess, ["serve"], "./evaluation_model")
    
    # Congeliamo il grafo
    frozen_graph = tf.compat.v1.graph_util.convert_variables_to_constants(
        sess, sess.graph.as_graph_def(), ["dense_2/Sigmoid"]
    )
    
    # Salviamo il grafo congelato in formato .pb
    with tf.gfile.GFile("./evaluation_model/frozen_model.pb", "wb") as f:
        f.write(frozen_graph.SerializeToString())

INFO:tensorflow:Restoring parameters from ./evaluation_model\variables\variables
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.
